In [60]:
import random
import os

words = ['Environment', 'Archives', 'Pronounciation', 
         'Hour', 'Wednesday', 'Violence', 'Tomb', 'Suite', 
         'Iron', 'Reciept', 'Chores']
random_words = random.sample(words, 5)
random_string = ' '.join(random_words)
print(random_string)

# Directory path to traverse
directory_path = 'models/'
models = []

for word in random_words:
    for file in os.listdir(directory_path):
        if word in file:
            models.append(file)

# model = joblib.load("models/" + models[0])
# model.predict()


Chores Suite Hour Tomb Reciept


In [21]:
import sounddevice as sd
import librosa
import ast
import pandas as pd

def extract_mfcc(audio_array, sample_rate):
    mfccs = librosa.feature.mfcc(y=audio_array, sr=sample_rate)
    return mfccs.tolist()

freq = 44100
duration = 2
recording = sd.rec(int(duration * freq), samplerate=freq, channels=2, dtype='float32')
sd.wait()  # Wait for the recording to complete
y = recording[:, 0]  # Use only one channel if you have stereo recording
sr = freq

mfcc_features = extract_mfcc(y, sr)

# Create DataFrame with a single column named 'MFCC'
dfMFCC = pd.DataFrame({"MFCC": mfcc_features})
# print(dfMFCC)
dfMFCC["MFCC"] = dfMFCC["MFCC"].apply(ast.literal_eval)

ValueError: malformed node or string: [-609.4856567382812, -609.4856567382812, -609.4856567382812, -609.4856567382812, -609.4856567382812, -609.4856567382812, -609.4856567382812, -609.4856567382812, -604.679931640625, -590.4852294921875, -559.720947265625, -520.3655395507812, -489.3622131347656, -465.9929504394531, -452.8149719238281, -435.890380859375, -415.26190185546875, -395.27752685546875, -383.06427001953125, -363.6819152832031, -347.570068359375, -339.5436706542969, -329.9855041503906, -315.2740478515625, -306.6613464355469, -302.5282897949219, -296.8883056640625, -299.37384033203125, -299.468017578125, -295.77325439453125, -292.1350402832031, -299.0865478515625, -304.3124694824219, -308.5631103515625, -305.0271911621094, -307.48974609375, -306.3254699707031, -306.1604919433594, -299.4473876953125, -294.5365905761719, -293.49212646484375, -296.596923828125, -294.16192626953125, -302.6404113769531, -303.5140075683594, -300.5515441894531, -304.7602233886719, -301.3059387207031, -2

In [18]:
import sounddevice as sd
import librosa
import ast
import pandas as pd
def extract_mfcc(audio_array, sample_rate):
    mfccs = librosa.feature.mfcc(y=audio_array, sr=sample_rate)
    return mfccs.tolist()

freq = 44100
duration = 2
recording = sd.rec(int(duration * freq), samplerate=freq, channels=2, dtype='float32')
sd.wait()  # Wait for the recording to complete
y = recording[:, 0]  # Use only one channel if you have stereo recording
sr = freq

mfcc_features = extract_mfcc(y, sr)
print(len(mfcc_features))
mfcc_features = mfcc_features.apply(ast.literal_eval)
mfcc_features

ValueError: malformed node or string: [[-774.1679077148438, -774.1679077148438, -774.1679077148438, -774.1679077148438, -774.1679077148438, -774.1679077148438, -774.1679077148438, -768.8052978515625, -746.7252197265625, -698.9524536132812, -649.1610107421875, -603.7162475585938, -563.268798828125, -524.28515625, -488.6083679199219, -461.8927307128906, -429.6945495605469, -404.1799621582031, -389.5199890136719, -366.7716369628906, -351.4461364746094, -340.29248046875, -328.2919616699219, -316.31585693359375, -304.031005859375, -300.1056823730469, -299.10662841796875, -296.6037902832031, -298.3864440917969, -302.0561218261719, -302.9696044921875, -302.7021484375, -305.8349304199219, -303.21405029296875, -302.8840637207031, -304.8682861328125, -303.1170959472656, -301.8288879394531, -298.6187438964844, -297.803466796875, -310.8716125488281, -304.9139709472656, -298.40399169921875, -298.2424011230469, -300.2910461425781, -304.915283203125, -303.1273498535156, -305.1188659667969, -304.23388

In [27]:
import sounddevice as sd
import librosa
import joblib
import ast
import pandas as pd
import numpy as np

def extract_mfcc(audio_array, sample_rate, expected_dim=1740):
    mfccs = librosa.feature.mfcc(y=audio_array, sr=sample_rate)
    flattened_mfcc = np.array(mfccs).flatten()
    
    # Pad or truncate the features to match the expected dimension
    if flattened_mfcc.shape[0] < expected_dim:
        # Pad with zeros if the dimension is less than expected
        padded_mfcc = np.pad(flattened_mfcc, (0, expected_dim - flattened_mfcc.shape[0]))
        return padded_mfcc.tolist()
    elif flattened_mfcc.shape[0] > expected_dim:
        # Truncate if the dimension is greater than expected
        truncated_mfcc = flattened_mfcc[:expected_dim]
        return truncated_mfcc.tolist()
    else:
        return flattened_mfcc.tolist()

freq = 44100
duration = 2
users = []

for i in range(len(random_words)):
    print(f'Please pronounce this word - {random_words[i]}')
    recording = sd.rec(int(duration * freq), samplerate=freq, channels=2, dtype='float32')
    sd.wait()  # Wait for the recording to complete
    y = recording[:, 0]  # Use only one channel if you have stereo recording
    sr = freq
    mfcc_features = extract_mfcc(y, sr)
    
    loaded_model = joblib.load("models/" + models[i])
    
    # Reshape the 1D array to 2D array
    mfcc_features_2d = np.array(mfcc_features).reshape(1, -1)
    
    user = loaded_model.predict(mfcc_features_2d)
    users.append(user)

print(users)

Please pronounce this word - Violence
Please pronounce this word - Wednesday
Please pronounce this word - Tomb
Please pronounce this word - Pronounciation
Please pronounce this word - Environment
[array(['Arun'], dtype=object), array(['Sunamdha'], dtype=object), array(['Arun'], dtype=object), array(['Abhishek'], dtype=object), array(['Sunamdha'], dtype=object)]


In [61]:
import sounddevice as sd
import librosa
import joblib
import ast
import pandas as pd
import numpy as np

def extract_mfcc(audio_array, sample_rate, expected_dim=1740):
    mfccs = librosa.feature.mfcc(y=audio_array, sr=sample_rate)
    
    # Flatten and balance features without padding
    flattened_mfcc = np.array(mfccs).flatten()
    if flattened_mfcc.shape[0] < expected_dim:
        # Repeat features to match the expected dimension
        balanced_mfcc = np.tile(flattened_mfcc, int(np.ceil(expected_dim / flattened_mfcc.shape[0])))
        return balanced_mfcc[:expected_dim].tolist()
    else:
        # Truncate if the dimension is greater than expected
        return flattened_mfcc[:expected_dim].tolist()

freq = 44100
duration = 2
users = []

for i in range(len(random_words)):
    print(f'Please pronounce this word - {random_words[i]}')
    recording = sd.rec(int(duration * freq), samplerate=freq, channels=2, dtype='float32')
    sd.wait()  # Wait for the recording to complete
    y = recording[:, 0]  # Use only one channel if you have stereo recording
    sr = freq
    mfcc_features = extract_mfcc(y, sr)
    
    loaded_model = joblib.load("models/" + models[i])
    
    # Ensure the dimensions match the model's expectations
    if len(mfcc_features) == 1740:  # Adjust this number based on your model's expectations
        # Reshape the 1D array to 2D array
        mfcc_features_2d = np.array(mfcc_features).reshape(1, -1)
        
        user = loaded_model.predict(mfcc_features_2d)
        users.append(user)
    else:
        print(f"Skipping prediction for {random_words[i]} due to incorrect feature dimensions.")

Please pronounce this word - Chores
Please pronounce this word - Suite
Please pronounce this word - Hour
Please pronounce this word - Tomb
Please pronounce this word - Reciept


In [62]:
users

[array(['Arun'], dtype=object),
 array(['Arun'], dtype=object),
 array(['Abhishek'], dtype=object),
 array(['Arun'], dtype=object),
 array(['Arun'], dtype=object)]